In [1]:
!nvidia-smi


Tue Aug 12 09:59:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   40C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install core libraries
!pip install -q -U "transformers==4.47.1" "datasets==3.2.0" "accelerate==1.2.1" \
               "trl==0.14.0" "peft==0.14.0" "huggingface_hub>=0.25.0" "math_verify==0.3.3"


In [3]:
from huggingface_hub import notebook_login
notebook_login()  


In [4]:
# Load the full test split of the dataset 
from datasets import load_dataset

dataset_id = "mohammad-shirkhani/social_movielens_compress"

test_dataset = load_dataset(dataset_id, split="test")
print(f"Loaded {dataset_id} — test split with {len(test_dataset)} rows.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/33.3M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/4.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/80000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Loaded mohammad-shirkhani/social_movielens_compress — test split with 10000 rows.


In [5]:
# Print dataset schema
print(test_dataset)


Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 10000
})


In [6]:
# Show the first raw row to understand field shapes/types
from pprint import pprint
pprint(test_dataset[0])


{'answer': 5.0,
 'item': {'Genres': 'Comedy',
          'MovieID': 153,
          'ReleaseDate': '01-Jan-1988',
          'Title': 'Fish Called Wanda, A (1988)'},
 'paths': ['user_question -> usersim -> User{UserID 643, Age 39, Gender Male, '
           'Occupation scientist} -> user_item_4 (rating=4) -> Item{MovieID '
           '234, Title "Jaws (1975)", Release Date 01-Jan-1975, Genres Action, '
           'Horror} -> itemsim -> Item{MovieID 423, Title "E.T. the '
           'Extra-Terrestrial (1982)", Release Date 01-Jan-1982, Genres '
           "Children's, Drama, Fantasy, Sci-Fi} -> itemsim -> item_question",
           'user_question -> user_item_3 (rating=3) -> Item{MovieID 826, Title '
           '"Phantom, The (1996)", Release Date 07-Jun-1996, Genres Adventure} '
           '-> item_user_2 (rating=2) -> User{UserID 207, Age 39, Gender Male, '
           'Occupation marketing} -> user_item_5 (rating=5) -> item_question',
           'user_question -> usersim -> User{UserID 30

In [7]:
# Create an English 'problem' field guiding the model to predict a rating with one decimal place
import json

def to_json_str(d):
    """Pretty and stable JSON string ."""
    return json.dumps(d or {}, ensure_ascii=False, sort_keys=False)

def build_problem(example):
    """
    Build an English prompt that:
    - Defines the task (predict rating with 0.1 precision in [1.0, 5.0]).
    - Shows user and item features.
    - Explains the bipartite graph and meta-paths.
    - Lists meta-paths as evidence.
    """
    user_block = to_json_str(example.get("user", {}))
    item_block = to_json_str(example.get("item", {}))

    paths_list = example.get("paths", []) or []
    if isinstance(paths_list, (list, tuple)):
        cleaned = [str(p).strip() for p in paths_list if p is not None]
        paths_str = "\n".join(f"- {p}" for p in cleaned)
    else:
        paths_str = f"- {str(paths_list).strip()}"

    prompt = (
        "You are a model that predicts the rating a user will give to a movie. "
        "Your task is to predict this user's rating for this movie in the range [1.0, 5.0] "
        "with a precision of one decimal place (e.g., 3.2 or 4.7). "
        "The result must be logical, evidence-based, and derived from the provided meta paths.\n\n"
        "Constraints and output format:\n"
        "• The rating must be between 1.0 and 5.0 (inclusive).\n"
        "• Output the rating with exactly one decimal place.\n"
        "• At the end, answer only one number (with one decimal place) as the 'Predicted Rating'.\n\n"
        "User features:\n"
        f"{user_block}\n\n"
        "Movie features:\n"
        f"{item_block}\n\n"
        "Reasoning guide:\n"
        "• The graph is bipartite (users–movies). Users similar to each other are connected with the 'usersim' edge, "
        "and similar movies are connected with the 'itemsim' edge.\n"
        "• Edges 'user_item_k' and 'item_user_k' indicate past ratings (rating=k) along the paths.\n"
        "• Use the paths below as positive/negative evidence; consider user similarity, movie similarity, genres, "
        "and rating patterns.\n"
        "• If evidence is contradictory, weigh the frequency and strength of the signals "
        "(path length, proximity, genre match, and rating patterns) to reach a conclusion.\n"
        "• After reasoning, round the final number to one decimal place within the allowed range.\n"
        "answer: Only a single number with one decimal place as the predicted rating.\n\n"
        "Meta Paths:\n"
        f"{paths_str}\n\n"
    )
    return {"problem": prompt}

# Apply to the whole test split
test_dataset = test_dataset.map(build_problem, desc="Building 'problem' prompts (test)")


Building 'problem' prompts (test):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
# Validate the newly created 'problem' field on the first row
from pprint import pprint
pprint(test_dataset[0])


{'answer': 5.0,
 'item': {'Genres': 'Comedy',
          'MovieID': 153,
          'ReleaseDate': '01-Jan-1988',
          'Title': 'Fish Called Wanda, A (1988)'},
 'paths': ['user_question -> usersim -> User{UserID 643, Age 39, Gender Male, '
           'Occupation scientist} -> user_item_4 (rating=4) -> Item{MovieID '
           '234, Title "Jaws (1975)", Release Date 01-Jan-1975, Genres Action, '
           'Horror} -> itemsim -> Item{MovieID 423, Title "E.T. the '
           'Extra-Terrestrial (1982)", Release Date 01-Jan-1982, Genres '
           "Children's, Drama, Fantasy, Sci-Fi} -> itemsim -> item_question",
           'user_question -> user_item_3 (rating=3) -> Item{MovieID 826, Title '
           '"Phantom, The (1996)", Release Date 07-Jun-1996, Genres Adventure} '
           '-> item_user_2 (rating=2) -> User{UserID 207, Age 39, Gender Male, '
           'Occupation marketing} -> user_item_5 (rating=5) -> item_question',
           'user_question -> usersim -> User{UserID 30

In [9]:
# Create the same system prompt used during training 
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

def make_conversation(example):
    """Wrap the 'problem' into a chat-style prompt with system+user messages."""
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }

test_dataset = test_dataset.map(make_conversation, desc="Building conversation prompts (test)")


Building conversation prompts (test):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
# Print the conversation-style prompt for the first test example
print(test_dataset[0]["prompt"])


[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>', 'role': 'system'}, {'content': 'You are a model that predicts the rating a user will give to a movie. Your task is to predict this user\'s rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.\n\nConstraints and output format:\n• The rating must be between 1.0 and 5.0 (inclusive).\n• Output the rating with exactly one decimal place.\n• At the end, answer only one number (with one decimal place) as the \'Predicted Rating\'.\n\nUser features:\n{"Us

In [11]:
print(test_dataset[0]["problem"])


You are a model that predicts the rating a user will give to a movie. Your task is to predict this user's rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.

Constraints and output format:
• The rating must be between 1.0 and 5.0 (inclusive).
• Output the rating with exactly one decimal place.
• At the end, answer only one number (with one decimal place) as the 'Predicted Rating'.

User features:
{"UserID": 758, "Age": 27, "Gender": "Male", "Occupation": "student"}

Movie features:
{"MovieID": 153, "Title": "Fish Called Wanda, A (1988)", "ReleaseDate": "01-Jan-1988", "Genres": "Comedy"}

Reasoning guide:
• The graph is bipartite (users–movies). Users similar to each other are connected with the 'usersim' edge, and similar movies are connected with the 'itemsim' edge.
• Edges 'user_item_k' and 'item_user_k' indicate past ratings (rating=k) along the 

In [12]:
print(test_dataset)


Dataset({
    features: ['user', 'item', 'answer', 'paths', 'problem', 'prompt'],
    num_rows: 10000
})


In [13]:
cols_to_drop = [c for c in ["user", "problem", "item", "paths"] if c in test_dataset.column_names]
test_dataset = test_dataset.remove_columns(cols_to_drop)
print(test_dataset)

from pprint import pprint
pprint({
    "answer": test_dataset[0]["answer"],
    "prompt": test_dataset[0]["prompt"],
})


Dataset({
    features: ['answer', 'prompt'],
    num_rows: 10000
})
{'answer': 5.0,
 'prompt': [{'content': 'A conversation between User and Assistant. The user '
                        'asks a question, and the Assistant solves it. The '
                        'assistant first thinks about the reasoning process in '
                        'the mind and then provides the user with the answer. '
                        'The reasoning process and answer are enclosed within '
                        '<think> </think> and <answer> </answer> tags, '
                        'respectively, i.e., <think> reasoning process here '
                        '</think><answer> answer here </answer>',
             'role': 'system'},
            {'content': 'You are a model that predicts the rating a user will '
                        "give to a movie. Your task is to predict this user's "
                        'rating for this movie in the range [1.0, 5.0] with a '
                        'prec

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_id    = "mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating-80000"

tokenizer = AutoTokenizer.from_pretrained(adapter_id, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
)

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

model = PeftModel.from_pretrained(
    model,
    adapter_id,
    torch_dtype="auto",
)

model.eval()
print("✅ Loaded base + LoRA adapter successfully. Vocab size:", len(tokenizer))


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

✅ Loaded base + LoRA adapter successfully. Vocab size: 151665


In [17]:
print(test_dataset["prompt"][0])

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>', 'role': 'system'}, {'content': 'You are a model that predicts the rating a user will give to a movie. Your task is to predict this user\'s rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.\n\nConstraints and output format:\n• The rating must be between 1.0 and 5.0 (inclusive).\n• Output the rating with exactly one decimal place.\n• At the end, answer only one number (with one decimal place) as the \'Predicted Rating\'.\n\nUser features:\n{"Us

In [23]:
import time


def generate_with_reasoning(prompt):
    # Build the prompt from the dataset
    prompt = " ".join(entry["content"] for entry in prompt)

    # Tokenize and move to the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text without gradients
    start_time = time.time()
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=2048)
    end_time = time.time()

    # Decode and extract model response
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Get inference time
    inference_duration = end_time - start_time

    # Get number of generated tokens
    num_input_tokens = inputs["input_ids"].shape[1]
    num_generated_tokens = output_ids.shape[1] - num_input_tokens

    return generated_text, inference_duration, num_generated_tokens

In [26]:
prompt = test_dataset["prompt"][3]
generated_text, inference_duration, num_generated_tokens = generate_with_reasoning(prompt)
print(generated_text)

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer> You are a model that predicts the rating a user will give to a movie. Your task is to predict this user's rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.

Constraints and output format:
• The rating must be between 1.0 and 5.0 (inclusive).
• Output the rating with exactly one decimal place.
• At the end, answer only one number (with one decimal place) as the 'Predicted Rating'.

User features:
{"UserID": 767, "Age": 70, "Gender": "Male", "Occupation": "eng